In [ ]:
import rasterio
import geopandas as gpd
import pyproj
import os
import matplotlib.pyplot as plt
import pandas as pd
import glob
import lightgbm as lgbm
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from numpy import *
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from cubist import Cubist
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit
root = r'C:/Users/Rasmus/Mit drev (thesis20212022@gmail.com)\Norway'
predict_root = root
#predict_root = 'C:/Users/Rasmus/Mit drev (thesis20212022@gmail.com)/Norway'

In [ ]:
def stack(path):
    VH2k = os.path.join(path , "VHyr2k.tif")
    CS2k = os.path.join(path , "CSyr2k.tif")
    VH2k_N = os.path.join(path , "VHyr2k_N.tif")
    CS2k_N = os.path.join(path , "CSyr2k_N.tif")

    
    ## ADD TO LIST
    listofmasks=[
                 #VH2k,
                 CS2k,
                 #VH2k_N,
                 CS2k_N,
                ]
    #
    print(listofmasks)
    
    lit_ravel = []
    i=0
    for rst in listofmasks:
        with rasterio.open(rst) as src:
            srcdata = src.read()
            print(srcdata.shape,'shape')
            ravel = srcdata[0].ravel()
            lit_ravel.append(ravel)
            profile = src.profile
            og_shape = srcdata.shape
            i=i+1
    
    print(len(lit_ravel))
    return lit_ravel, profile, og_shape

In [ ]:
def model(Training_Data_Path, Validation_Data_Path, Prediction_Path, outdir, date_pred): 
    
    #AOI = 'South'
    AOI = 'North'
    pts = gpd.read_file(Training_Data_Path)
    
    pts = pts.astype({"SD": float})
    pts = pts.astype({"VH2k": float})
    pts = pts.astype({"CS2k": float})
    #pts = pts.astype({"VH1k": float})
    #pts = pts.astype({"CS1k": float})
    
    
    print('pts', pts)
        # convert Series
    sd = pd.to_numeric(pts['SD'])

    # convert column "a" of a DataFrame
    pts['SD'] = pd.to_numeric(sd)
    
    #
    #pts = pts[pts['beaminfo'] == 'strong beam']
    pts = pd.DataFrame(pts.dropna())
    pts = pts[pts['SD'] < 6]
    pts = pts[pts['SD'] > 0]
    pts = pts[pts['VH2k'] > -2]
    pts = pts[pts['CS2k'] > -2]
    print('before drop nan:',pts.shape)
    pts = pd.DataFrame(pts.dropna())
     
    X = pts[['CS2k']]
    print(X.shape)
    y = pts['SD']
    print(y.shape)
    

    #Validation data
    
    pts_val = gpd.read_file(Training_Data_Path)
    
    pts_val = pts_val[pts_val['SD'] < 6]
    pts_val = pts_val[pts_val['SD'] > 0]
    pts_val = pts_val[pts_val['VH2k'] > -2]
    pts_val = pts_val[pts_val['CS2k'] > -2]
    
    in_array_val = np.array(pts_val.VH2k)
    pts_val['VH2k_T'] = np.array(1.2**in_array_val)   
    
    in_array2_val = np.array(pts_val.CS2k)
    pts_val['CS2k_T'] = np.array(0.8**in_array2_val)
    
    pts_val = pd.DataFrame(pts_val.dropna())
    X_val = pts_val[['CS2k']]
    y_val = pts_val['SD']
    
    
    #PolyRegression
    reg = make_pipeline(
    PolynomialFeatures(degree=(2)),
    LinearRegression(),)
    reg.fit(X, y)
    
    
    #LinearRegression
    #reg = LinearRegression().fit(X, y)
    
    ## LightGBM ##
    #reg = lgbm.LGBMRegressor(max_depth=50)
    #reg.fit(X, y)
    
    #SVR#
    #reg = SVR(kernel="poly", degree=2)
    #reg.fit(X, y)
    
    ## RF ##
    #reg = RandomForestRegressor(max_depth=5, random_state=0).fit(X_train,y_train)
    #eg = RandomForestRegressor(max_depth=5,random_state=0).fit(X,y)
    #importances = reg.feature_importances_
    #print('importance', importances)
    
    
    lit_ravel, profile, og_shape = stack(Prediction_Path)
    
    print(len(y_val))
    
    X_test = np.transpose(lit_ravel)
    if AOI == 'South':
        X_test = X_test[:,0:1]
    else:
        X_test = X_test[:,1:2]

    print(X_test.shape)
    X_test[np.isnan(X_test)] = -999
    print(X.shape)
    print(X_test.shape)
    y_pred = reg.predict(X_test)  # make predictions
    print('modelling done')    

    #Predict on training to calc R2
    y_pred_val = reg.predict(X_val)
    y_pred_train = reg.predict(X)
    
    #TRAINING STATS 
    r2 = r2_score(y, y_pred_train).round(2)
    r2 = format(r2, '.2f')
    print('r2_train', r2)
    
    rms = mean_squared_error(y, y_pred_train, squared=False).round(2)
    print('RMSE_train: ', rms)
    
    mae = mean_absolute_error(y, y_pred_train).round(2)
    print('MAE_train: ', mae)
    
    #VALIDATION STATS 
    r2_val = r2_score(y_val, y_pred_val).round(2)
    r2_val = format(r2_val, '.2f')    
    print('r2_val', r2_val)
    
    
    rms_val = mean_squared_error(y_val, y_pred_val, squared=False).round(2)
    print('RMSE_val: ', rms_val)
    
    mae_val = mean_absolute_error(y_val, y_pred_val).round(2)
    print('MAE_val: ', mae_val)
    
    Binary_Mask = Prediction_Path[:-23] + 'DEM/Binary_Mask_Norway_All_500m.tif'
    with rasterio.open(Binary_Mask) as mask:
            srcdata = mask.read()
            print('ndval',mask.nodata)
            B_Mask = srcdata[0].ravel()
    y_pred = y_pred * B_Mask
    
    y_pred[y_pred==0] = np.nan
    y_pred[y_pred>10] = np.nan
    y_pred[y_pred<-10] = np.nan
    y_pred[(y_pred<0) & (y_pred>-10)] = -999
    y_pred = y_pred.reshape(og_shape)
    
    with rasterio.open(outdir + date_pred + 'test.tif', 'w', **profile) as dst:
        dst.nodata = np.nan
        dst.write(y_pred.astype(rasterio.float64))
        
    return y_pred_val,y_val, y_pred_train, y, X

In [ ]:
rootdir = 'C:/Users/Rasmus/Mit drev (thesis20212022@gmail.com)/Norway/Dates_pred/'
dates_pred_lit=[] 
for file in os.listdir(rootdir):
    dates_pred_lit.append(file)  
print(dates_pred_lit)
for date_pred in dates_pred_lit[:-20]: 
    #SD_CONCAT_3 IS THE RIGHT ONE
    #Forkerte datoer: 20210421, 20210220, 20210503, 20210527, 20211223
    Training_Data_Path = root+'/Training_Data/SD_concat_3.shp'
    Validation_Data_Path = root+'/Training_Data/SD_concat_val.shp'
    Prediction_Path = predict_root+'/Dates_pred/'+ date_pred +'/S1/'
    outdir = root+'/Results_full/'
    y_pred_val,y_val,y_pred_train,y, X = model(Training_Data_Path, Validation_Data_Path,Prediction_Path, outdir, date_pred)

In [ ]:
plt.figure(figsize=(5, 5))
plt.title('model snow depths')
plt.xlabel('model predicted snow depth (meter)')
plt.ylabel('icesat-2 snow_depth (meter)')
plt.scatter(y_pred_train, y, color='green', s=1);